In [1]:
from IPython.display import Markdown as md

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
inputFile="./S288C_Expression/EmpiricalExp/S288C.datatable_0.3Top_0.3Bot.csv"
df=pd.read_csv(inputFile)
print(df[:3])

   Unnamed: 0   GeneName                                           Sequence  \
0        1941  YHR079C-A  ATGAACTATTTGGAAACACAGTTAAATAAAAAGCAAAAACAGATAC...   
1        4329    YPL121C  ATGCACAATCAGGAAGAGTGGCTAGACAAGGACAAAACTTTGGTGA...   
2        1496    YGL170C  ATGGGAGCTGGCACTCTTTTAAATGGATTAGAAAAGGAAAACTTTC...   

         EXP  Label  
0  59.599320      0  
1  59.995476      0  
2  62.303841      0  


After loading the data, the next step is to convert the raw coding sequences to text sentences compsed of codon usage ranks.

In [4]:
#Start

In [5]:
from Bio import SeqIO
import sys
# insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, './CUB_Code')
import CodonLibraries as CL

seqList=df["Sequence"]
sentenceList=[]
for seq in seqList:
    codonList = CL.loadSequence(seq)
    sentence=""
    for codon in codonList:
        sentence+=codon+" "
    sentenceList.append(sentence[:-1])#this removes the extra redundant space in the end

Now we have converted the sequence to codonLists, then concated them into string sentences 

In [6]:
print(sentenceList[:1])

['AAC TAT TTG GAA ACA CAG TTA AAT AAA AAG CAA AAA CAG ATA CAG GAA TAC GAA AGT ATG AAT GGC AAC CTG ATA AAG ATG TTT GAG CAA TTG TCT AAA GAA AAG AAA AAT GAT GAG ACA CCA AAA AAA ATT TCC TCG ACG TAC ATT AAA GAG TTA AAG GAG TAC AAC GAA TTG AGA GAT GCC GGT TTA AGG TTG GCC CAA ATA ATT GCT GAT GAA AAG CAA TGC AAA ATT AAG GAT GTT TTT GAA GAG ATC GGT TAT TCA ATG AAG GAC']


The next step is to get the genome wide codon usage and generate codon ranks, which will used to replace the codon strings

In [7]:
import FastaFileToRankSequences as FFTRS
import os

inputFile="./Data/s288c.fasta" # this is the genome file used to generate codon ranks, there are other approaches to generatesuch ranks
outputFile="./s288cRank.txt"
print(inputFile)
FFTRS.process(inputFile,outputFile)

./Data/s288c.fasta
Selected id Type: locus_tag
There are 0 entries NOT found out of 5990
5990 distinct record in 5990 entries
0  sequences are not length of three


We have an intermediate file in the name specified by {{outputFile}}, this can also be used for training the model and tokenizer, but we will apply one more operation on this file, composing single codon ranks into words of size k

In [11]:
def convertLineToWords(line,n):
    line= line.replace(' ','')
    wordList=[]
    for i in range(0,len(line)-n,n):
        wordList.append(line[i:i+n])
    sentence=" ".join(wordList)
    return sentence



In [9]:
f=open(outputFile)
k=4 #k specifies the wordsize, aka number of codon ranks in a word, or understood as size of codon windows
lines=f.readlines()

In [12]:
sentenceList=[]
for line in lines:
    sentenceList.append(convertLineToWords(line,4))

In [13]:
print(len(sentenceList))
print(len(sentenceList))

5990
